In [6]:
! pip install transformers
! pip install sentencepiece
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00


In [7]:
import random
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from tqdm import tqdm
import torch
import numpy as np
import time
import pandas as pd

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

### reading the test data

In [3]:
!gdown 1MjK8syScLm4EpTL_4PUCXUjQqGuYHZtY

Downloading...
From: https://drive.google.com/uc?id=1MjK8syScLm4EpTL_4PUCXUjQqGuYHZtY
To: /content/test.jsonl
100% 399k/399k [00:00<00:00, 71.2MB/s]


In [4]:
import json

with open('test.jsonl', 'r') as json_file:
    json_list = list(json_file)

questions = {'math_and_logic':[],'common_knowledge':[],'literature':[]}
for json_str in json_list:
    result = json.loads(json_str)
    questions[result['category']].append({'question':result['question'],'candidates':result['candidates'],'answer':result['answer']})
    #print(f"result: {result}")


In [5]:
for key in questions:
  print(key,len(questions[key]))

math_and_logic 350
common_knowledge 350
literature 350


In [6]:
random.seed(42)
samples = {}
samples['math_and_logic'] = random.sample(questions['math_and_logic'], k=200)
samples['common_knowledge'] = random.sample(questions['common_knowledge'], k=200)
samples['literature'] = random.sample(questions['literature'], k=200)

In [7]:
for key in samples:
  print(key)
  print(samples[key][0])

math_and_logic
{'question': 'عدد ۶ یک عدد کامل نامیده می\u200cشود، زیرا برابر است با مجموع مقسوم علیه\u200cهایش به استثنای خود عدد ۶ کدامیک از عددهای زیر عدد کامل است؟', 'candidates': ['۱۲', '۲۸', '۳۶', '۱۶'], 'answer': '2'}
common_knowledge
{'question': ' از ابتدای تاریخ، کدام گزینه زیر انسانها را به همکاری با یکدیگر واداشته است؟', 'candidates': ['ضرورتهای زندگی اجتماعی', 'اهداف سیاسی دولت- شهرها', 'تشکیل اجتماعات انسانی', 'فرهنگ پذیری'], 'answer': '1'}
literature
{'question': 'كدام كلمه جمع است', 'candidates': ['پهلوان', 'مازندران', 'يزدان', 'ديوان'], 'answer': '4'}


In [8]:
sample_qmath = [sample['question'] for sample in samples['math_and_logic']]
sample_qknowledge = [sample['question'] for sample in samples['common_knowledge']]
sample_qliterature = [sample['question'] for sample in samples['literature']]

In [9]:
others_math = [q for q in questions['math_and_logic'] if (q['question'] not in sample_qmath)]
others_knowledge = [q for q in questions['common_knowledge'] if (q['question'] not in sample_qknowledge)]
others_literature = [q for q in questions['literature'] if (q['question'] not in sample_qliterature)]

In [10]:
random.seed(42)
one_shot_math = random.sample(others_math, k=1)
one_shot_knowledge = random.sample(others_knowledge, k=1)
one_shot_literature = random.sample(others_literature, k=1)

three_shot_math = random.sample(others_math, k=3)
three_shot_knowledge = random.sample(others_knowledge, k=3)
three_shot_literature = random.sample(others_literature, k=3)

In [11]:
one_shot_math

[{'question': 'حاصل عبارت ۴ + ۵۵۳ برابر است با ؟',
  'candidates': ['558', '557', '556', '554'],
  'answer': '2'}]

In [12]:
one_shot_knowledge

[{'question': 'وسیع ترین کشور جهان کدام است؟',
  'candidates': ['آمریکا', 'کانادا', 'روسیه', 'چین'],
  'answer': '3'}]

In [13]:
one_shot_literature

[{'question': 'در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟',
  'candidates': ['سرود رگبار، دستچين، عبور، چمن لاله',
   'چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار',
   'در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور',
   'تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن'],
  'answer': '1'}]

## test on fine-tuned models

In [ ]:
def run_model(model,tokenizer,input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt").to(DEVICE)
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    #print(output)
    return output


models = ['persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice',
          'persiannlp/mt5-base-parsinlu-multiple-choice',
          'persiannlp/mt5-large-parsinlu-multiple-choice',
          'persiannlp/mt5-small-parsinlu-multiple-choice',
          'persiannlp/mt5-small-parsinlu-arc-comqa-obqa-multiple-choice',
          'persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice'
          ]

In [ ]:
results = {}

### math and logics

In [ ]:
results['math_and_logic'] = {}
for model_name in models:
  print(f'________________________________ {model_name} ________________________________')
  results['math_and_logic'][model_name] = []
  #model_name = f"persiannlp/mt5-{model_size}-parsinlu-arc-comqa-obqa-multiple-choice"
  tokenizer = MT5Tokenizer.from_pretrained(model_name,return_tensors="pt")
  model = MT5ForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
  for data in tqdm(samples['math_and_logic']):
    input_string = " <sep> ".join([data['question']]+data['candidates'])
    output = run_model(model,tokenizer,input_string)
    results['math_and_logic'][model_name].append({'question':data['question'],'answer':data['candidates'][int(data['answer'])-1],'model_ans':output[0]})
  del model
  del tokenizer
  torch.cuda.empty_cache()


________________________________ persiannlp/mt5-base-parsinlu-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/200 [00:13<44:26, 13.40s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 200/200 [00:33<00:00,  6.03it/s]


________________________________ persiannlp/mt5-large-parsinlu-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:36<00:00,  5.41it/s]


________________________________ persiannlp/mt5-small-parsinlu-multiple-choice ________________________________


...

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


100%|██████████| 200/200 [00:36<00:00,  5.44it/s]


________________________________ persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


100%|██████████| 200/200 [00:20<00:00,  9.73it/s]


In [ ]:
math_and_logic_accuracy = {}
for key in results['math_and_logic']:
  math_and_logic_accuracy[key] = np.array([1 if r['answer']==r['model_ans'] else 0 for r in results['math_and_logic'][key] ]).mean()
math_and_logic_accuracy

{'persiannlp/mt5-base-parsinlu-multiple-choice': 0.365,
 'persiannlp/mt5-large-parsinlu-multiple-choice': 0.395,
 'persiannlp/mt5-small-parsinlu-multiple-choice': 0.385,
 'persiannlp/mt5-small-parsinlu-arc-comqa-obqa-multiple-choice': 0.35,
 'persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice': 0.36,
 'persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice': 0.38}

### common knowledge

In [ ]:
results['common_knowledge'] = {}
for model_name in models:
  print(f'________________________________ {model_name} ________________________________')
  results['common_knowledge'][model_name] = []
  #model_name = f"persiannlp/mt5-{model_size}-parsinlu-arc-comqa-obqa-multiple-choice"
  tokenizer = MT5Tokenizer.from_pretrained(model_name,return_tensors="pt")
  model = MT5ForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
  for data in tqdm(samples['common_knowledge']):
    input_string = " <sep> ".join([data['question']]+data['candidates'])
    output = run_model(model,tokenizer,input_string)
    results['common_knowledge'][model_name].append({'answer':data['candidates'][int(data['answer'])-1],'model_ans':output[0]})
  del model
  del tokenizer
  torch.cuda.empty_cache()

________________________________ persiannlp/mt5-base-parsinlu-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/200 [00:09<30:47,  9.29s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 200/200 [00:44<00:00,  4.53it/s]


________________________________ persiannlp/mt5-large-parsinlu-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:57<00:00,  3.47it/s]


________________________________ persiannlp/mt5-small-parsinlu-multiple-choice ________________________________


...

________________________________ persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:59<00:00,  3.38it/s]


________________________________ persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


100%|██████████| 200/200 [00:32<00:00,  6.19it/s]


In [ ]:
common_knowledge_accuracy = {}
for key in results['common_knowledge']:
  common_knowledge_accuracy[key] = np.array([1 if r['answer']==r['model_ans'] else 0 for r in results['common_knowledge'][key]]).mean()
common_knowledge_accuracy

{'persiannlp/mt5-base-parsinlu-multiple-choice': 0.24,
 'persiannlp/mt5-large-parsinlu-multiple-choice': 0.245,
 'persiannlp/mt5-small-parsinlu-multiple-choice': 0.235,
 'persiannlp/mt5-small-parsinlu-arc-comqa-obqa-multiple-choice': 0.23,
 'persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice': 0.25,
 'persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice': 0.215}

### literature

In [ ]:
results['literature'] = {}
for model_name in models:
  print(f'________________________________ {model_name} ________________________________')
  results['literature'][model_name] = []
  #model_name = f"persiannlp/mt5-{model_size}-parsinlu-arc-comqa-obqa-multiple-choice"
  tokenizer = MT5Tokenizer.from_pretrained(model_name,return_tensors="pt")
  model = MT5ForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
  for data in tqdm(samples['literature']):
    input_string = " <sep> ".join([data['question']]+data['candidates'])
    output = run_model(model,tokenizer,input_string)
    results['literature'][model_name].append({'answer':data['candidates'][int(data['answer'])-1],'model_ans':output[0]})
  del model
  del tokenizer
  torch.cuda.empty_cache()

________________________________ persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/200 [00:09<30:24,  9.17s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 200/200 [01:25<00:00,  2.35it/s]


________________________________ persiannlp/mt5-base-parsinlu-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:41<00:00,  4.79it/s]


________________________________ persiannlp/mt5-large-parsinlu-multiple-choice ________________________________


...

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:28<00:00,  7.02it/s]


________________________________ persiannlp/mt5-small-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:28<00:00,  7.08it/s]


________________________________ persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice ________________________________


You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.
100%|██████████| 200/200 [00:41<00:00,  4.79it/s]


In [ ]:
literature_accuracy = {}
for key in results['literature']:
  literature_accuracy[key] = np.array([1 if r['answer']==r['model_ans'] else 0 for r in results['literature'][key]]).mean()
literature_accuracy

{'persiannlp/mt5-large-parsinlu-arc-comqa-obqa-multiple-choice': 0.275,
 'persiannlp/mt5-base-parsinlu-multiple-choice': 0.315,
 'persiannlp/mt5-large-parsinlu-multiple-choice': 0.3,
 'persiannlp/mt5-small-parsinlu-multiple-choice': 0.335,
 'persiannlp/mt5-small-parsinlu-arc-comqa-obqa-multiple-choice': 0.295,
 'persiannlp/mt5-base-parsinlu-arc-comqa-obqa-multiple-choice': 0.31}

In [14]:
import openai
openai.api_key  = "..."

def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## math and logic prompt

english - zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(143,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 156


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  question:
  '''شهرام از بهرام کوچکتر است. مهرام از بهرام بزرگتر است. شهرام از مهرام کوچکتر است. اگر جمله های اول و دوم درست باشد، جمله سوم:'''

  candidates:
  '''[به طور قطع درست است.,به طور قطع نادرست است.,ممکن است درست نباشد.,ممکن نیست درست باشد.]'''

  answer:
  
model ans:  ممکن است درست نباشد.
correct ans:  به طور قطع درست است.
-------------------------------------------------------------------------------------
question 157


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  question:
  '''معدل دریافتی یک روز ۷ نفر کارمند یک فروشگاه ۷۵۰ ریال است. هیچیک از آنها بیشتر از ۸۰۰ ریال دریافت نمی‌کنند کمترین مبلغ دریافتی یک نفر، چند ریال می‌تواند ب

APIError: ignored

In [ ]:
df = pd.DataFrame(results)
df.to_csv('mc-english-prompt-zero-57-144.csv')

english - 1shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_math)
print(sample_string)


  question:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  candidtaes:
  [558,557,556,554]

  answer:
  557


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

Streaming output truncated to the last 5000 lines.
  answer:
  
model ans:  ۳۵
correct ans:  ۳۵
-------------------------------------------------------------------------------------
question 17


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  candidtaes:
  [558,557,556,554]

  answer:
  557

  question:
  '''150 تا 4 تا برابر است با ......'''

  candidates:
  '''[300,500,600,900]'''

  answer:
  
model ans:  600
correct ans:  600
-------------------------------------------------------------------------------------
question 18


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  candidtae

In [ ]:
df = pd.DataFrame(results)
df.to_csv('mathmc-english-prompt-1shot.csv')

english - 3shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_math)
print(sample_string)


  question:
  شعاع دایره اي یک دهم افزایش یافته است. مساحت مربع محاطی آن چند درصد افزایش مییابد؟

  candidtaes:
  [۷۹,۲۱,۱۹,۸۱]

  answer:
  ۲۱

  question:
  %40 عدد 100 برابر است با ....

  candidtaes:
  [30,40,50,60]

  answer:
  40

  question:
  حاصل عبارت ۴ + ۵ برابر است با ؟

  candidtaes:
  [9,10,11,8]

  answer:
  9


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(168,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 169


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  شعاع دایره اي یک دهم افزایش یافته است. مساحت مربع محاطی آن چند درصد افزایش مییابد؟

  candidtaes:
  [۷۹,۲۱,۱۹,۸۱]

  answer:
  ۲۱

  question:
  %40 عدد 100 برابر است با ....

  candidtaes:
  [30,40,50,60]

  answer:
  40

  question:
  حاصل عبارت ۴ + ۵ برابر است با ؟

  candidtaes:
  [9,10,11,8]

  answer:
  9

  question:
  '''عده دانش آموزان دختر سه برابر عده دانش آموزان پسر دریک کلاس می‌باشد. کدام یک از عددهای زیر نمی‌تواند عده دانش آموزان این کلاس را نشان دهد؟'''

  candidates:
  '''[۲۴,۳۲,۲۶,۳۶]'''

  answer:
  
model ans:  ۳۶
correct ans:  ۲۶
-------------------------------------------------------------------------------------
question 170


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the quest

In [ ]:
df = pd.DataFrame(results)
df.to_csv('mathmc-english-prompt-3shot-169-200.csv')

In [ ]:
files = ['mathmc-english-prompt-3shot-169-200.csv']
df = pd.read_csv('mathmc-english-prompt-3shot-1-169.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index(drop=True).to_csv('mathmc-english-prompt-3shot.csv')

persian - zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 1


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''عدد ۶ یک عدد کامل نامیده می‌شود، زیرا برابر است با مجموع مقسوم علیه‌هایش به استثنای خود عدد ۶ کدامیک از عددهای زیر عدد کامل است؟'''

  گزینه ها:
  '''[۱۲,۲۸,۳۶,۱۶]'''

  جواب:
  
model ans:  گزینه ۳۶
correct ans:  ۲۸
-------------------------------------------------------------------------------------
question 2


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''حاصل عبارت ۲ - ۵ برابر است با ؟'''

  گزینه ها:
  '''[2,3,4,5]'''

  جواب:
  
model ans:  -3
correct ans:  3
-------------------------------------------------------------------------------------
question 3


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود ب

In [ ]:
df = pd.DataFrame(results)
df.to_csv('mathmc-persian-prompt-zero.csv')

persian - 1shot

In [ ]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_math)
print(sample_string)


  سوال:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  گزینه ها:
  [558,557,556,554]

  جواب:
  557


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(192,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 193


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  گزینه ها:
  [558,557,556,554]

  جواب:
  557

  سوال:
  '''اگر 23 تیر ماه چهارشنبه باشد، پنجم مرداد ماه همان سال کدام روز هفته است؟'''

  گزینه ها:
  '''[سه شنبه,چهارشنبه,دوشنبه,جمعه]'''

  جواب:
  
model ans:  دوشنبه
correct ans:  سه شنبه
-------------------------------------------------------------------------------------
question 194


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  حاصل عبارت ۴ + ۵۵۳ برابر است با ؟

  گزینه ها:
  [558,557,556,554]

  جواب:
  557

  سوال:
  '''چند درصد ۵۰۰۰ برابر ۵ می‌شود؟'''

  گزینه ها:
  '''[۱۰۰,۱,یک دهم,یک صدم]'''

  جواب:
  
model ans:  یک صدم
correct ans:  ۱
----

In [ ]:
df = pd.DataFrame(results)
df.to_csv('mathmc-persian-prompt-1shot-193-200.csv')

In [ ]:
files = ['mathmc-persian-prompt-1shot-30-104.csv',
         'mathmc-persian-prompt-1shot-104-167.csv',
         'mathmc-persian-prompt-1shot-167-193.csv',
         'mathmc-persian-prompt-1shot-193-200.csv']
df = pd.read_csv('mathmc-persian-prompt-1shot-1-30.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index(drop = True).to_csv('mathmc-persian-prompt-1shot.csv')

persian - 3shot

In [ ]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_math)
print(sample_string)


  سوال:
  شعاع دایره اي یک دهم افزایش یافته است. مساحت مربع محاطی آن چند درصد افزایش مییابد؟

  گزینه ها:
  [۷۹,۲۱,۱۹,۸۱]

  جواب:
  ۲۱

  سوال:
  %40 عدد 100 برابر است با ....

  گزینه ها:
  [30,40,50,60]

  جواب:
  40

  سوال:
  حاصل عبارت ۴ + ۵ برابر است با ؟

  گزینه ها:
  [9,10,11,8]

  جواب:
  9


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(199,200):
  sample = samples['math_and_logic'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 200


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  شعاع دایره اي یک دهم افزایش یافته است. مساحت مربع محاطی آن چند درصد افزایش مییابد؟

  گزینه ها:
  [۷۹,۲۱,۱۹,۸۱]

  جواب:
  ۲۱

  سوال:
  %40 عدد 100 برابر است با ....

  گزینه ها:
  [30,40,50,60]

  جواب:
  40

  سوال:
  حاصل عبارت ۴ + ۵ برابر است با ؟

  گزینه ها:
  [9,10,11,8]

  جواب:
  9

  سوال:
  '''منبع ۷۸۰ ،A لیتر آب بیشتر از منبع B دارد به وسیله یک مجرا ۱۲۰ لیتر آب منبع A وارد منبع B می‌شود
'''

  گزینه ها:
  '''[۶۶۰ لیتر,۵۴۰ لیتر,۹۰۰ لیتر,۷۲۰ لیتر]'''

  جواب:
  
model ans:  ۶۶۰ لیتر
correct ans:  ۵۴۰ لیتر
-------------------------------------------------------------------------------------


In [ ]:
df = pd.DataFrame(results)
df.to_csv('mathmc-persian-prompt-3shot-200.csv')

In [ ]:
files = ['mathmc-persian-prompt-3shot-200.csv']
df = pd.read_csv('mathmc-persian-prompt-3shot-1-199.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index(drop = True).to_csv('mathmc-persian-prompt-3shot.csv')

## common knowledge prompt

english zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

english - 1shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_knowledge)
print(sample_string)


  question:
  وسیع ترین کشور جهان کدام است؟

  candidtaes:
  [آمریکا,کانادا,روسیه,چین]

  answer:
  روسیه


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

Streaming output truncated to the last 5000 lines.
  
model ans:  ده روز
correct ans:  ده روز
-------------------------------------------------------------------------------------
question 16


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  وسیع ترین کشور جهان کدام است؟

  candidtaes:
  [آمریکا,کانادا,روسیه,چین]

  answer:
  روسیه

  question:
  '''نخستین جنگ دوران خلافت امام علی (ع) چه نام داشت؟'''

  candidates:
  '''[جمل,صفین,نهروان,خوارج]'''

  answer:
  
model ans:  صفین
correct ans:  جمل
-------------------------------------------------------------------------------------
question 17


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  وسیع ترین کشور جها

In [ ]:
df = pd.DataFrame(results)
df.to_csv('commonmc-english-prompt-1shot.csv')

english - 3shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_knowledge)
print(sample_string)


  question:
  کدام یک از موارد زیر، عکس جریان ارتباط اصلی است و طی آن گیرنده پیام جدیدی را می فرستد؟

  candidtaes:
  [ورود عوامل مزاحم, کد گذاری,استفاده از نماد,بازخورد ]

  answer:
  بازخورد 

  question:
  برنامه های عمرانی کشور چگونه است؟

  candidtaes:
  [بلند مدت,کوتاه مدت,میان مدت,بلند مدت و میان مدت]

  answer:
  میان مدت

  question:
  حکومت قاجار در چه سالی به پایان رسید؟

  candidtaes:
  [۱۳۰۷ ش,۱۳۲۰ ش,۱۳۱۷ ش,۱۳۰۴ ش]

  answer:
  ۱۳۰۴ ش


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(178,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 179


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  کدام یک از موارد زیر، عکس جریان ارتباط اصلی است و طی آن گیرنده پیام جدیدی را می فرستد؟

  candidtaes:
  [ورود عوامل مزاحم, کد گذاری,استفاده از نماد,بازخورد ]

  answer:
  بازخورد 

  question:
  برنامه های عمرانی کشور چگونه است؟

  candidtaes:
  [بلند مدت,کوتاه مدت,میان مدت,بلند مدت و میان مدت]

  answer:
  میان مدت

  question:
  حکومت قاجار در چه سالی به پایان رسید؟

  candidtaes:
  [۱۳۰۷ ش,۱۳۲۰ ش,۱۳۱۷ ش,۱۳۰۴ ش]

  answer:
  ۱۳۰۴ ش

  question:
  '''کدام قوم می پنداشت بیماری را ناشی از ورود ارواح خبیث در بدن می دانست؟'''

  candidates:
  '''[سومر,آشور,ایلام,مصر]'''

  answer:
  
model ans:  سومر
correct ans:  سومر
-------------------------------------------------------------------------------------
question 180


  In this task, you will be presented with a 

In [ ]:
df = pd.DataFrame(results)
df.to_csv('commonmc-english-prompt-3shot-179-200.csv')

In [ ]:
files = ['commonmc-english-prompt-3shot-179-200.csv']
df = pd.read_csv('commonmc-english-prompt-3shot-1-179.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index().to_csv('commonmc-english-prompt-3shot.csv')

persian - zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(118,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)


question 119


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''علت حملات متعدد آشوریان به مسکن قبایل ساکن در زاگرس چه بوده است؟'''

  گزینه ها:
  '''[گسترش قلمرو,تصرف منابع آب,نیاز به چراگاه,نیاز به مناع فلز]'''

  جواب:
  
model ans:  گسترش قلمرو
correct ans:  نیاز به مناع فلز
-------------------------------------------------------------------------------------
question 120


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  '''آخرین فرمانده سپاه اسلامی در حیات پیامبر (ص) چه کسی بود؟'''

  گزینه ها:
  '''[زید بن حارثه,خالد بن ولید,اسامه بن زید,عبدالله بن دواحه]'''

  جواب:
  
model ans:  عبدالله بن دواحه
correct ans:  اسامه بن زید
-------------------------------------------------------------------------------------
question 12

In [ ]:
df = pd.DataFrame(results)
df.to_csv('commonmc-persian-prompt-118-200-zero.csv')

In [ ]:
files = ['commonmc-persian-prompt-118-200-zero.csv']
df = pd.read_csv('commonmc-persian-prompt-1-119-zero.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index().to_csv('commonmc-persian-prompt-zero.csv')

persian - 1shot

In [ ]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_knowledge)
print(sample_string)


  سوال:
  وسیع ترین کشور جهان کدام است؟

  گزینه ها:
  [آمریکا,کانادا,روسیه,چین]

  جواب:
  روسیه


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(64,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 65


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  وسیع ترین کشور جهان کدام است؟

  گزینه ها:
  [آمریکا,کانادا,روسیه,چین]

  جواب:
  روسیه

  سوال:
  '''کهن ترین پول فلزی جهان در کدام منطقه پیدا شد؟'''

  گزینه ها:
  '''[تخت جمشید,شوش,لودیه,موهنجودارو]'''

  جواب:
  
model ans:  تخت جمشید
correct ans:  موهنجودارو
-------------------------------------------------------------------------------------
question 66


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  وسیع ترین کشور جهان کدام است؟

  گزینه ها:
  [آمریکا,کانادا,روسیه,چین]

  جواب:
  روسیه

  سوال:
  '''کتاب هملت را چه کسی نوشته؟'''

  گزینه ها:
  '''[داوینچی,شکسپیر,کافکا,]'''

  جواب:
  
model ans:  شکسپیر
correct ans:  شکسپیر
---------

In [ ]:
df = pd.DataFrame(results)
df.to_csv('commonmc-persian-prompt-1shot-65-200.csv')

In [ ]:
files = ['commonmc-persian-prompt-1shot-37-48.csv',
         'commonmc-persian-prompt-1shot-48-65.csv',
         'commonmc-persian-prompt-1shot-65-200.csv']
df = pd.read_csv('commonmc-persian-prompt-1shot-1-37.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index(drop=True).to_csv('commonmc-persian-prompt-1shot.csv')

persian-3 shot

In [ ]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_knowledge)
print(sample_string)


  سوال:
  کدام یک از موارد زیر، عکس جریان ارتباط اصلی است و طی آن گیرنده پیام جدیدی را می فرستد؟

  گزینه ها:
  [ورود عوامل مزاحم, کد گذاری,استفاده از نماد,بازخورد ]

  جواب:
  بازخورد 

  سوال:
  برنامه های عمرانی کشور چگونه است؟

  گزینه ها:
  [بلند مدت,کوتاه مدت,میان مدت,بلند مدت و میان مدت]

  جواب:
  میان مدت

  سوال:
  حکومت قاجار در چه سالی به پایان رسید؟

  گزینه ها:
  [۱۳۰۷ ش,۱۳۲۰ ش,۱۳۱۷ ش,۱۳۰۴ ش]

  جواب:
  ۱۳۰۴ ش


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(190,200):
  sample = samples['common_knowledge'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 191


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  کدام یک از موارد زیر، عکس جریان ارتباط اصلی است و طی آن گیرنده پیام جدیدی را می فرستد؟

  گزینه ها:
  [ورود عوامل مزاحم, کد گذاری,استفاده از نماد,بازخورد ]

  جواب:
  بازخورد 

  سوال:
  برنامه های عمرانی کشور چگونه است؟

  گزینه ها:
  [بلند مدت,کوتاه مدت,میان مدت,بلند مدت و میان مدت]

  جواب:
  میان مدت

  سوال:
  حکومت قاجار در چه سالی به پایان رسید؟

  گزینه ها:
  [۱۳۰۷ ش,۱۳۲۰ ش,۱۳۱۷ ش,۱۳۰۴ ش]

  جواب:
  ۱۳۰۴ ش

  سوال:
  '''استفاده از آجرهای لعاب دار نخستین بار توسط چه کسانی ابداع شد؟'''

  گزینه ها:
  '''[ماد,هخامنشیان,ایلام,مصریان]'''

  جواب:
  
model ans:  مصریان
correct ans:  ماد
-------------------------------------------------------------------------------------
question 192


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خو

In [ ]:
df = pd.DataFrame(results)
df.to_csv('commonmc-persian-prompt-3shot-191-200.csv')

In [ ]:
files = ['commonmc-persian-prompt-3shot-191-200.csv']
df = pd.read_csv('commonmc-persian-prompt-3shot-1-191.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)
df.reset_index(drop=True).to_csv('commonmc-persian-prompt-3shot.csv')

## literature prompt

english - zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

english - 1shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_literature)
print(sample_string)


  question:
  در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟

  candidtaes:
  [سرود رگبار، دستچين، عبور، چمن لاله,چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار,در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور,تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن]

  answer:
  سرود رگبار، دستچين، عبور، چمن لاله


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

Streaming output truncated to the last 5000 lines.
  در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟

  candidtaes:
  [سرود رگبار، دستچين، عبور، چمن لاله,چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار,در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور,تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن]

  answer:
  سرود رگبار، دستچين، عبور، چمن لاله

  question:
  '''بیت زیر، با کدام بیت، قرابت مفهومی دارد؟
«لاف از سخن چو در توان زد   آن خشت بود که پر توان زد»
'''

  candidates:
  '''[ سخن را روی در صاحبدلان است    نگویند از حرم الا به محرم,چو یافوت باید سخن بی‌زبان   سبک سنگ لیکن بهایش گران,یاران سخن راست همین است که گفتم    در باغ جهان هرچه بکارید برآید, سخن پیش فرهنگیان سخته گوی    به هر کس نوازنده و تازه‌روی]'''

  answer:
  
model ans:  سخن را روی در صاحبدلان است
correct ans:  چو یافوت باید سخن بی‌زبان   سبک سنگ لیکن بهایش گران
-------------------------------------------------------------------------------------
question 18


  In this task, you will be presented 

In [ ]:
df = pd.DataFrame(results)
df.to_csv('lmc-english-prompt-1shot.csv')

english - 3shot

In [ ]:
template = f"""
  question:
  Q

  candidtaes:
  C

  answer:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_literature)
print(sample_string)


  question:
  در عيد .   .   . دين اسلام كامل شد.

  candidtaes:
  [عيد فطر,عيد قربان,عيد مبعث,عيد غدير]

  answer:
  عيد غدير

  question:
  کدام گزینه ازموضوعات شعری عصررودکی نیست؟

  candidtaes:
  [وصف,عرفان,مدح,اندرز]

  answer:
  عرفان

  question:
  آنچه که از ارزش واقعی چیزی بکاهد :؟

  candidtaes:
  [انتقاد,شایعه,فراوانی,نقص]

  answer:
  نقص


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(189,200):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  {sample_string}
  question:
  '''{question}'''

  candidates:
  '''[{",".join(candidates)}]'''

  answer:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 190


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your knowledge. choose the answer from the given candidates.

  sample:
  
  question:
  در عيد .   .   . دين اسلام كامل شد.

  candidtaes:
  [عيد فطر,عيد قربان,عيد مبعث,عيد غدير]

  answer:
  عيد غدير

  question:
  کدام گزینه ازموضوعات شعری عصررودکی نیست؟

  candidtaes:
  [وصف,عرفان,مدح,اندرز]

  answer:
  عرفان

  question:
  آنچه که از ارزش واقعی چیزی بکاهد :؟

  candidtaes:
  [انتقاد,شایعه,فراوانی,نقص]

  answer:
  نقص

  question:
  '''« در پوستین خلق افتادن » کنایه از.......... است.'''

  candidates:
  '''[دروغ گفتن,صحبت کردن,غیبت کردن,گدایی کردن]'''

  answer:
  
model ans:  غیبت کردن
correct ans:  غیبت کردن
-------------------------------------------------------------------------------------
question 191


  In this task, you will be presented with a multiple-choice question in Persian, and you should answer the question based on your kno

In [ ]:
df = pd.DataFrame(results)
df.to_csv('lmc-english-prompt-3shot-189-200.csv')

In [ ]:
files = ['lmc-english-prompt-3shot-189-200.csv']
df = pd.read_csv('lmc-english-prompt-3shot-1-189.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.to_csv('lmc-english-prompt-3shot.csv')

persian - zero

In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(129,130):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')
  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  {question}

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)


question 130


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.

  سوال:
  عبارات زیر، با همه ابیات قرابت مفهومی دارد؛ به‌جز:
«آن روزش بکشتند و دیگر روز بسوختند و سوم روزش به باد بردادند؛ یعنی، عشق این است.»

  گزینه ها:
  '''[ به آب تیغ خون عاشقان از جوش ننشیند    همان گل‌بانگ وحدت از لب منصور می‌بارد,چون اوحدی در کوی دل تا من شنیدم بوی دل    هرجا که کردم روی دل پیروز و منصور آمدم, با اهل فنا دارد هرکس سر یکرنگی    باید که به رنگ شمع از رفتن سر خندد,شد سر منصور آخر گوی چوگان فنا    میوه چون شد پخته خود را از شجر می‌افکند]'''

  جواب:
  
model ans:  «به آب تیغ خون عاشقان از جوش ننشیند»
correct ans:  چون اوحدی در کوی دل تا من شنیدم بوی دل    هرجا که کردم روی دل پیروز و منصور آمدم
-------------------------------------------------------------------------------------


In [ ]:
df = pd.DataFrame(results)
df.to_csv('lmc-persian-prompt-zero-130.csv')

In [ ]:
files = ['lmc-persian-prompt-zero-46-130.csv','lmc-persian-prompt-zero-130.csv',
         'lmc-persian-prompt-zero-130-161.csv','lmc-persian-prompt-zero-161-191.csv','lmc-persian-prompt-zero-191-200.csv']
df = pd.read_csv('lmc-persian-prompt-zero-1-46.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)

In [ ]:
df.reset_index(drop = True).to_csv('lmc-persian-prompt-zero.csv')

persian - 1shot

In [ ]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in one_shot_literature)
print(sample_string)


  سوال:
  در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟

  گزینه ها:
  [سرود رگبار، دستچين، عبور، چمن لاله,چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار,در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور,تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن]

  جواب:
  سرود رگبار، دستچين، عبور، چمن لاله


In [ ]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(0,200):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 51


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  در کدام گزینه، آثار موسوی گرما رودی تماما درست است؟

  گزینه ها:
  [سرود رگبار، دستچين، عبور، چمن لاله,چمن لاله، خطّ خون، مثل درخت در شب باران، سرود رگبار,در سايه سار نخل ولايت، از بودن و سرودن، خطّ خون، عبور,تاناكجا، دستچين، در سايه سار نخل ولايت، از بودن و سرودن]

  جواب:
  سرود رگبار، دستچين، عبور، چمن لاله

  سوال:
  '''رابطه ي آشکار با پنهان مثل رابطه ي:'''

  گزینه ها:
  '''[معلوم است با مجهول,روز است با شب,روشن است با تاریک,خیر است با شر]'''

  جواب:
  
model ans:  روشن است با تاریک
correct ans:  معلوم است با مجهول
-------------------------------------------------------------------------------------
question 52


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید

RateLimitError: ignored

In [ ]:
df = pd.DataFrame(results)
df.to_csv('lmc-persian-prompt-1shot-1-51.csv')

persian - 3shot

In [16]:
template = f"""
  سوال:
  Q

  گزینه ها:
  C

  جواب:
  A
"""

sample_string = ''.join(template.replace('Q',s['question']).replace('A',s['candidates'][int(s['answer'])-1]).replace('C',f"""[{",".join(s['candidates'])}]""") for s in three_shot_literature)
print(sample_string)


  سوال:
  در عيد .   .   . دين اسلام كامل شد.

  گزینه ها:
  [عيد فطر,عيد قربان,عيد مبعث,عيد غدير]

  جواب:
  عيد غدير

  سوال:
  کدام گزینه ازموضوعات شعری عصررودکی نیست؟

  گزینه ها:
  [وصف,عرفان,مدح,اندرز]

  جواب:
  عرفان

  سوال:
  آنچه که از ارزش واقعی چیزی بکاهد :؟

  گزینه ها:
  [انتقاد,شایعه,فراوانی,نقص]

  جواب:
  نقص


In [21]:
results = {"question":[],"correct_ans":[],"model_ans":[]}
for i in range(141,200):
  sample = samples['literature'][i]

  question = sample['question']
  candidates = sample['candidates']
  correct_ans =  candidates[int(sample['answer'])-1]
  print(f'question {i+1}')

  prompt = f"""

  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  {sample_string}
  سوال:
  '''{question}'''

  گزینه ها:
  '''[{",".join(candidates)}]'''

  جواب:
  """

  response = get_completion(prompt)
  print(prompt)
  print("model ans: ",response)
  print("correct ans: ",correct_ans)
  print("-------------------------------------------------------------------------------------")

  results['question'].append(question)
  results['correct_ans'].append(correct_ans)
  results['model_ans'].append(response)

  time.sleep(22)

question 142


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  در عيد .   .   . دين اسلام كامل شد.

  گزینه ها:
  [عيد فطر,عيد قربان,عيد مبعث,عيد غدير]

  جواب:
  عيد غدير

  سوال:
  کدام گزینه ازموضوعات شعری عصررودکی نیست؟

  گزینه ها:
  [وصف,عرفان,مدح,اندرز]

  جواب:
  عرفان

  سوال:
  آنچه که از ارزش واقعی چیزی بکاهد :؟

  گزینه ها:
  [انتقاد,شایعه,فراوانی,نقص]

  جواب:
  نقص

  سوال:
  '''بسیار به شلوغ مانند کم است به :؟'''

  گزینه ها:
  '''[اندک,خلوت,گران,فراوان]'''

  جواب:
  
model ans:  اندک
correct ans:  خلوت
-------------------------------------------------------------------------------------
question 143


  در ادامه، به شما یک سوال چند گزینه‌ای به زبان فارسی نشان داده می شود. شما باید بر اساس دانش خود به سوال پاسخ دهید. پاسخ خود را از بین گزینه‌های داده شده انتخاب کنید.


  نمونه:
  
  سوال:
  در عيد .   .   . دين اسلام كامل شد.

 

In [22]:
df = pd.DataFrame(results)
df.to_csv('lmc-persian-prompt-3shot-142-200.csv')

In [23]:
files = ['lmc-persian-prompt-3shot-27-118.csv','lmc-persian-prompt-3shot-118-142.csv','lmc-persian-prompt-3shot-142-200.csv']
df = pd.read_csv('lmc-persian-prompt-3shot-1-27.csv',index_col=[0])
for file in files:
  df1 = pd.read_csv(file,index_col=[0])
  df = pd.concat([df, df1], axis=0)
df.reset_index(drop=True).to_csv('lmc-persian-prompt-3shot.csv')

## gpt evaluate

In [12]:
!unzip multiple_choice.zip

Archive:  multiple_choice.zip
   creating: multiple_choice/
  inflating: multiple_choice/commonmc-persian-prompt-3shot.csv  
  inflating: __MACOSX/multiple_choice/._commonmc-persian-prompt-3shot.csv  
  inflating: multiple_choice/commonmc-english-prompt-3shot.csv  
  inflating: __MACOSX/multiple_choice/._commonmc-english-prompt-3shot.csv  
  inflating: multiple_choice/.DS_Store  
  inflating: __MACOSX/multiple_choice/._.DS_Store  
  inflating: multiple_choice/mathmc-persian-prompt-zero.csv  
  inflating: __MACOSX/multiple_choice/._mathmc-persian-prompt-zero.csv  
  inflating: multiple_choice/commonmc-english-prompt-1shot.csv  
  inflating: __MACOSX/multiple_choice/._commonmc-english-prompt-1shot.csv  
  inflating: multiple_choice/commonmc-english-prompt-zero.csv  
  inflating: __MACOSX/multiple_choice/._commonmc-english-prompt-zero.csv  
  inflating: multiple_choice/commonmc-persian-prompt-1shot.csv  
  inflating: __MACOSX/multiple_choice/._commonmc-persian-prompt-1shot.csv  
  inflati

In [13]:
import os

files = os.listdir('/content/multiple_choice')

In [14]:
files

['commonmc-english-prompt-1shot.csv',
 'mathmc-persian-prompt-3shot.csv',
 'commonmc-english-prompt-zero.csv',
 'commonmc-english-prompt-3shot.csv',
 'mathmc-persian-prompt-zero.csv',
 'mathmc-english-prompt-3shot.csv',
 'lmc-persian-prompt-3shot.csv',
 'mathmc-persian-prompt-1shot.csv',
 'lmc-english-prompt-3shot.csv',
 'commonmc-persian-prompt-1shot.csv',
 'lmc-english-prompt-zero.csv',
 'commonmc-persian-prompt-3shot.csv',
 'commonmc-persian-prompt-zero.csv',
 'mathmc-english-prompt-1shot.csv',
 'mathmc-english-prompt-zero.csv',
 '.DS_Store',
 'lmc-english-prompt-1shot.csv',
 'lmc-persian-prompt-zero.csv',
 'lmc-persian-prompt-1shot.csv']

In [16]:
for file_name in files:
  if ("prompt" in file_name):
    df = pd.read_csv(f'/content/multiple_choice/{file_name}')
    accuracy = np.array([1 if df.loc[i,'correct_ans']==df.loc[i,'model_ans'] else 0 for i in range(len(df))]).mean()
    print(f"{file_name[:-4]} : {accuracy}")

commonmc-english-prompt-1shot : 0.43
mathmc-persian-prompt-3shot : 0.435
commonmc-english-prompt-zero : 0.425
commonmc-english-prompt-3shot : 0.43
mathmc-persian-prompt-zero : 0.45
mathmc-english-prompt-3shot : 0.415
lmc-persian-prompt-3shot : 0.275
mathmc-persian-prompt-1shot : 0.45
lmc-english-prompt-3shot : 0.29
commonmc-persian-prompt-1shot : 0.395
lmc-english-prompt-zero : 0.31
commonmc-persian-prompt-3shot : 0.445
commonmc-persian-prompt-zero : 0.385
mathmc-english-prompt-1shot : 0.435
mathmc-english-prompt-zero : 0.445
lmc-english-prompt-1shot : 0.305
lmc-persian-prompt-zero : 0.28
lmc-persian-prompt-1shot : 0.295
